# ¿Cuál es tu mascota?

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import os
from torch.utils.data import DataLoader

In [2]:
data_dir = './dataset'
train_dir = os.path.join(data_dir, 'training_set')
test_dir = os.path.join(data_dir, 'test_set')

#### Resumen General de las Transformaciones

transforms.Compose:
Combina múltiples transformaciones en una secuencia.
Argumento: una lista de objetos de transformación.

transforms.RandomResizedCrop (solo en entrenamiento):
Recorta y redimensiona aleatoriamente la imagen.
Parámetro: tamaño del recorte.

transforms.RandomHorizontalFlip (solo en entrenamiento):
Invierte la imagen horizontalmente con una probabilidad del 50%.

transforms.Resize (solo en prueba):
Redimensiona la imagen manteniendo la relación de aspecto.
Parámetro: tamaño del lado más corto.


transforms.CenterCrop (solo en prueba):
Recorta el centro de la imagen.
Parámetro: tamaño del recorte.
Centro de la Imagen: CenterCrop intentará recortar una sección de 224x224 píxeles del centro de la imagen de 500x800. Esto funcionará si la imagen es suficientemente grande. Sin embargo, si la imagen es más pequeña que 224x224 píxeles en algún lado, este recorte no funcionará correctamente y puede resultar en errores o en imágenes recortadas inadecuadamente.

transforms.ToTensor:
Convierte la imagen en un tensor.

transforms.Normalize:
Normaliza los valores de los píxeles.
Parámetros: listas de medias y desviaciones estándar para cada canal.

#### Resumen de las Diferencias
Entrenamiento:

Objetivo: Aumentar la variabilidad y la robustez del modelo.
Transformaciones: Incluyen aumentación de datos como RandomResizedCrop y RandomHorizontalFlip.
Prueba:

Objetivo: Evaluar el rendimiento del modelo de manera justa y consistente.
Transformaciones: No incluyen aumentación de datos. Solo aseguran que las imágenes sean del tamaño adecuado y estén normalizadas.

In [3]:
# trasnforms.Compose is an object
# transforms.RandomResizedCrop() is an object
train_transforms = transforms.Compose([   
    transforms.RandomResizedCrop(224), # Crop a random region of an image and then resize to a fix size 224 is the size of the crop
    transforms.RandomHorizontalFlip(), # Flips the image with a probability of 0.5
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Al asegurar que el lado más corto de todas las imágenes sea de 256 píxeles, nos garantizamos de que haya
# suficiente área en la imagen para hacer un recorte central de 224x224 píxeles. 
test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224), 
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # List of means and list of std (one per channel RGB)
])

In [4]:

# Importamos y aplicamos las transformaciones a cada imagen
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transforms)
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transforms)

print(train_dataset)
print(type(train_dataset))
# Hacemos batches
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

print(train_loader)
print(type(train_loader))


Dataset ImageFolder
    Number of datapoints: 8000
    Root location: ./dataset/training_set
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=True)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
<class 'torchvision.datasets.folder.ImageFolder'>
<class 'torch.utils.data.dataloader.DataLoader'>


## BUILD CNN

Primera Capa Convolucional:

Entrada: 3 canales (RGB).
Salida: 32 filtros.
Tamaño del filtro: 3x3.
Stride: 1.
Segunda Capa Convolucional:

Entrada: 32 filtros (de la primera capa).
Salida: 64 filtros.
Tamaño del filtro: 3x3.
Stride: 1.
Seguido por Max Pooling con tamaño de kernel 2x2.
Tercera Capa Convolucional:

Entrada: 64 filtros (de la segunda capa).
Salida: 128 filtros.
Tamaño del filtro: 3x3.
Stride: 1.
Seguido por Max Pooling con tamaño de kernel 2x2.
Capas Completamente Conectadas:

La salida de las capas convolucionales se aplana y pasa por una capa completamente conectada con 512 unidades.
La capa final tiene 2 unidades para las 2 clases (perros y gatos).

In [12]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.conv3 = nn.Conv2d(64, 128, 3, 1)
        self.fc1 = nn.Linear(128 * 54 * 54, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2)
        x = x.view(-1, 128 * 54 * 54)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)


optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [13]:
num_epochs = 10
log_interval = 10

model.train()
for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

Train Epoch: 0 [0/8000 (0%)]	Loss: 0.694132
Train Epoch: 0 [320/8000 (4%)]	Loss: 0.706841
Train Epoch: 0 [640/8000 (8%)]	Loss: 0.685812
Train Epoch: 0 [960/8000 (12%)]	Loss: 0.697628
Train Epoch: 0 [1280/8000 (16%)]	Loss: 0.701136
Train Epoch: 0 [1600/8000 (20%)]	Loss: 0.691285
Train Epoch: 0 [1920/8000 (24%)]	Loss: 0.692559
Train Epoch: 0 [2240/8000 (28%)]	Loss: 0.695069
Train Epoch: 0 [2560/8000 (32%)]	Loss: 0.689085
Train Epoch: 0 [2880/8000 (36%)]	Loss: 0.689786
Train Epoch: 0 [3200/8000 (40%)]	Loss: 0.692772
Train Epoch: 0 [3520/8000 (44%)]	Loss: 0.686974
Train Epoch: 0 [3840/8000 (48%)]	Loss: 0.693434
Train Epoch: 0 [4160/8000 (52%)]	Loss: 0.678756
Train Epoch: 0 [4480/8000 (56%)]	Loss: 0.681976
Train Epoch: 0 [4800/8000 (60%)]	Loss: 0.705909
Train Epoch: 0 [5120/8000 (64%)]	Loss: 0.677231
Train Epoch: 0 [5440/8000 (68%)]	Loss: 0.698271
Train Epoch: 0 [5760/8000 (72%)]	Loss: 0.688481
Train Epoch: 0 [6080/8000 (76%)]	Loss: 0.688406
Train Epoch: 0 [6400/8000 (80%)]	Loss: 0.695982
T

KeyboardInterrupt: 

In [ ]:
n_epochs = 5

for epoch in range(1, n_epochs + 1):
    train(model, device, train_loader, optimizer, criterion, epoch)
    test(model, device, test_loader, criterion)